<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Compartments" data-toc-modified-id="Compartments-1">Compartments</a></span></li><li><span><a href="#TADs" data-toc-modified-id="TADs-2">TADs</a></span><ul class="toc-item"><li><span><a href="#Save-Chromosome-object-(with-TAD-definition)" data-toc-modified-id="Save-Chromosome-object-(with-TAD-definition)-2.1">Save Chromosome object (with TAD definition)</a></span></li></ul></li></ul></div>

# Compartments

Compartments are detected on the full genome matrix

In [2]:
from pytadbit.parsers.hic_parser import load_hic_data_from_bam
from pytadbit.parsers.genome_parser import get_gc_content, parse_fasta
from pytadbit.utils.extraviews import nicer 
from cPickle import load

In [3]:
cell   = 'mouse_B'

In [4]:
reso = 200000
base_path = 'results/fragment/{0}_both/03_filtering/valid_reads12_{0}.bam'
bias_path = 'results/fragment/{1}_both/04_normalizing/biases_{0}_{1}.pick'
bads_path = 'results/fragment/{1}_both/04_normalizing/bad_columns_{0}_{1}.pick'

In [5]:
def my_load_hic_data(rep, reso):
    hic_data = load_hic_data_from_bam(base_path.format(rep), resolution=reso, ncpus=8)
    hic_data.bias = load(open(bias_path.format(rep, nicer(reso, ''))))
    hic_data.bads = load(open(bads_path.format(rep, nicer(reso, ''))))
    return hic_data

In [6]:
hic_data = my_load_hic_data(cell, reso)


  (Matrix size 13641x13641)                                                    [2018-09-18 18:30:15]

  - Parsing BAM (122 chunks)                                                   [2018-09-18 18:30:15]
     .......... .......... .......... .......... ..........     50/122
     .......... .......... .......... .......... ..........    100/122
     .......... .......... ..                                  122/122

  - Getting matrices                                                           [2018-09-18 18:30:37]
     .......... .......... .......... .......... ..........     50/122
     .......... .......... .......... .......... ..........    100/122
     .......... .......... ..                                  122/122



IOError: [Errno 2] No such file or directory: 'results/fragment/200kb_both/04_normalizing/biases_mouse_B_200kb.pick'

In [ ]:
! mkdir -p results/fragment/$cell\_both/05_segmenting

In [ ]:
rich_in_A = get_gc_content(parse_fasta('genome/Mus_musculus-GRCm38.p6/Mus_musculus-GRCm38.p6.fa'), 
                           by_chrom=True ,resolution=reso, n_cpus=8)

In [ ]:
crm = 'chr3'
corr = hic_data.find_compartments(
        show=True, crms=[crm], vmin=-0.15, vmax=0.15, rich_in_A=rich_in_A,
        savedata='results/fragmesnt/{0}_both/05_segmenting/compartments_{1}_{2}.tsv'.format(cell, crm, reso))

In [ ]:
crm = 'chr3'
corr = hic_data.find_compartments(
        show=True, crms=[crm], vmin=-0.15, vmax=0.15, rich_in_A=rich_in_A,
        savedata='results/fragment/{0}_both/05_segmenting/compartments_{1}_{2}.tsv'.format(cell, crm, reso))

In [ ]:
! head -n 20 results/fragment/mouse_PSC_both/05_segmenting/compartments_chr3_300000.tsv

In [ ]:
! head -n 20 results/fragment/mouse_B_both/05_segmenting/compartments_chr3_300000.tsv

# TADs

In [ ]:
from pytadbit import Chromosome

In [ ]:
chrname = 'chr19'
crm = Chromosome(chrname)
crm.add_experiment('mouse_B', 
                   hic_data='results/fragment/mouse_B_both/04_normalizing/100000_raw/%s.mat' % (chrname), 
                   norm_data='results/fragment/mouse_B_both/04_normalizing/100000_raw/%s.mat' % (chrname),
                   resolution=100000)
crm.add_experiment('mouse_PSC', 
                   hic_data='results/fragment/mouse_PSC_both/04_normalizing/100000_raw/%s.mat' % (chrname),
                   norm_data='results/fragment/mouse_PSC_both/04_normalizing/100000_raw/%s.mat' % (chrname),
                   resolution=100000)

In [ ]:
crm.visualize([('mouse_B', 'mouse_PSC')])

In [ ]:
crm.visualize([('mouse_B', 'mouse_PSC')], normalized=True)

In [ ]:
crm.find_tad(['mouse_B', 'mouse_PSC'], n_cpus=8)

In [ ]:
crm.visualize([('mouse_B', 'mouse_PSC')], normalized=True, paint_tads=True, )

In [ ]:
crm.visualize([('mouse_B', 'mouse_PSC')], normalized=True, paint_tads=True, focus=(300, 400))

In [ ]:
B = crm.experiments['mouse_B']
PSC = crm.experiments['mouse_PSC']

In [ ]:
crm.experiments

## Save Chromosome object (with TAD definition)

In [ ]:
crm.save_chromosome('results/fragment/chr3.tdb')